# Get the best prices for used books at ebay and some resellers

In [ ]:
import isbnlib
import pandas as pd
from tqdm import tqdm

In [ ]:
with open('Buecher_ISBN.csv') as f:
    isbns = f.read()[1:]

In [ ]:
data = []
for isbn in tqdm(isbns.split('\n')):
    try:
        book = isbnlib.meta(isbn.replace('-', ''))
        data.append(book)
    except:
        pass

In [ ]:
data = [x for x in data if x]
df_1 = pd.DataFrame(data)
df_1.head()

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
p = re.compile('\d+,\d+')

In [ ]:
data = []
for isbn, title in tqdm(df_1[['ISBN-13', 'Title']].values):
    query = isbn+ ' ' +title
    
    url = 'https://www.ebay.de/sch/i.html?_nkw={}&_in_kw=1&_ex_kw=&_sacat=0&LH_Sold=1&_udlo=&_udhi=&_samilow=&_samihi=&_sadis=10&_fpos=&LH_SALE_CURRENCY=0&_sop=12&_dmd=1&_ipg=50&LH_Complete=1'.format(query)
    bs = BeautifulSoup(requests.get(url).text, 'lxml')
    
    prices = []
    try:
        for elem in bs.find('ul', {'id':'ListViewInner'}).findAll('li'):
            if 'id' in elem.attrs:
                price = elem.find('span', {'class':'bold bidsold'}).get_text().strip()
                price_float = float(re.search(p, price)[0].replace(',', '.'))
                prices.append(price_float)

            if elem.find('div', {'class':'kand-expansion kand-expansion--list-view'}):
                break
    except:
        pass
    if prices:
        prices = np.mean(prices)
    else:
        prices = 0.0
    data.append([isbn, prices])

In [ ]:
df_2 = pd.DataFrame(data, columns=['ISBN-13', 'Ebay'])
df_2.head()

In [ ]:
df_ebay = df_1.merge(df_2, on='ISBN-13', how='outer')
df_ebay.head()

In [ ]:
from selenium import webdriver

In [ ]:
url = 'https://www.bonavendi.de/verkaufen/sammeleingabe.html'

In [ ]:
driver = webdriver.Firefox()
driver.get(url)

In [ ]:
isbn_elem = driver.find_element_by_xpath("//textarea")
isbn_elem.clear()
isbn_elem.send_keys(isbns)

In [ ]:
driver.find_element_by_xpath("//button[@class='btn btn-primary']").click()

In [ ]:
url = 'https://www.bonavendi.de/verkaufen/korb.html'
driver.get(url)

Manuell Preise aktualisieren

In [ ]:
bs = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
driver.close()

In [ ]:
data = []
for elem in bs.find('div', {'class':'bv-cart-body bv-cart-items-body col-md-9'}).find('tbody').findAll('tr'):
    art_isbn = elem.find('p', {'class':'bv-barcode'}).get_text()
    try:
        art_buyer = elem.find('td', {'class':'bv-cart-items-partner hidden-sm hidden-xs'}).find('span').get_text()
    except:
        art_buyer = 'Missing'
    try:
        art_price = elem.find('div', {'class':'bv-cart-items-price'}).find('span').get_text().split('\xa0')[0].replace(',', '.')
    except:
        art_price = '0'
    data.append([art_isbn, art_buyer, art_price])

In [ ]:
df_3 = pd.DataFrame(data, columns=['ISBN-13', 'Best_Buy', 'Best_Price'])

In [ ]:
df = df_ebay.merge(df_3, on='ISBN-13', how='outer').fillna(0)

In [ ]:
for col in df.columns:
    df[col] = df[col].astype('float', errors='ignore')

In [ ]:
df.info()

In [ ]:
df.sort_values('Ebay', ascending=False)[['Title', 'Ebay']]

In [ ]:
df.sort_values('Best_Price', ascending=False)[['Title', 'Best_Price']]